In [47]:
import numpy as np 
from matplotlib import pyplot as plt 
import seaborn as sns 
from tqdm import tqdm 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier  , AdaBoostClassifier , GradientBoostingClassifier
from sklearn.metrics import precision_score , recall_score 
import xgboost as xgb 
import pandas as pd 
import sklearn.neighbors._base
from os import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest 
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay
sns.set_style('whitegrid')
%load_ext autoreload
%autoreload 2
sns.set_style('whitegrid')
from IPython.display import display
from imblearn.over_sampling import SMOTE

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
def details(data_sent , comments = ''):
    if(comments):
        print(comments)
    sp = (data_sent.isna().sum().sum())/ data_sent.size
    print('________________________________________________')
    print('Sparsity in the data : {:.2f}'.format(sp))
    num_rows = data_sent.shape
    print('Data shape' , num_rows)
    #display(data_sent['class'].value_counts())
    print('Number of sources : ')
    cl = data_sent['class'].unique()
    for c in cl:
        num_src = len(data_sent[data_sent['class']==c]['src_id'].unique())
        num_obs = len(data_sent[data_sent['class']==c])
        print(c ,' \t ' , num_src , '\t' , num_obs)

In [49]:
from features import phot_flux , en_flux , hard, powlaw_fit , bb_fit , brems_fit , intra_obs_var , inter_ob_var , info_col , phot_flux_hilim , phot_flux_lolim , en_flux_hilim , en_flux_lolim

In [60]:
info_col_cl = info_col + ['class']
from features import small_ind_subset
feat_to_use = info_col_cl + small_ind_subset 
feat_to_use.remove('var_inter_sigma_u')
#feat_to_use

In [61]:
data_imp_v2 = pd.read_csv('imputed_data/new-data-no-lolim.csv' , index_col='obs_id')
#starting_with_small_subset
#data_imp_v2 = pd.read_csv('imputed_data/small-feat-subset-new-data.csv', index_col='obs_id')
data_imp_v2 = data_imp_v2[feat_to_use]

In [62]:
details(data_imp_v2)

________________________________________________
Sparsity in the data : 0.00
Data shape (3750, 28)
Number of sources : 
CV  	  282 	 2044
PL  	  232 	 1104
LX  	  74 	 602


In [63]:

df_comb = pd.DataFrame()
for s in data_imp_v2['src_id'].unique()[:]:
    temp = data_imp_v2[data_imp_v2['src_id']==s]
    temp_id =  temp[info_col_cl]
    temp_val = temp.drop(columns=info_col_cl)
    temp_mean = temp_val.mean().to_frame().T
    temp_id_top = temp_id.iloc[0:1].reset_index()
    temp_final = pd.concat([temp_id_top , temp_mean] , axis=1).set_index('obs_id')
    df_comb = df_comb.append(temp_final)
display(df_comb)


,src_id,num_obs,src_n,name,ra,dec,livetime,significance,likelihood,class,...,flux_aper_hilim_u,flux_aper_lolim_h,var_inter_index_h,flux_aper_lolim_s,var_mean_b,var_sigma_b,ks_prob_b,var_prob_b,var_inter_index_s,var_inter_index_m
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,25,CSS120424:125906+242634,2CXO J004247.1+411619,10.696540,41.27215,4103.7,52.93,16618.73000,CV,...,13.180832,15.145466,7.895,15.390043,0.003259,0.000823,0.602003,0.532361,5.00,5.446667
CV_1_obs_0,CV_1,12,W32,2CXO J002405.6-720449,6.023632,-72.08036,31673.8,20.78,1834.51100,CV,...,14.322629,14.626767,5.000,14.664618,0.002208,0.000588,0.610771,0.577386,5.00,0.000000
CV_2_obs_0,CV_2,8,CXOGlb_J002405.2-720446,2CXO J002405.2-720446,6.021793,-72.07968,5544.8,9.72,548.75650,CV,...,14.443047,15.212169,0.000,15.274571,0.002153,0.000477,0.688203,0.516768,0.00,4.000000
CV_3_obs_0,CV_3,13,Cl__NGC_104_MAM_M54898,2CXO J002406.0-720456,6.025117,-72.08228,65237.5,25.99,3275.90600,CV,...,14.358313,14.169467,6.000,14.376190,0.003670,0.000332,0.388990,0.233054,6.00,8.000000
CV_4_obs_0,CV_4,13,CXOGlb_J002404.9-720455,2CXO J002404.9-720455,6.020589,-72.08210,31673.8,19.64,2291.21000,CV,...,14.509655,14.387837,8.000,15.122289,0.003253,0.000536,0.613627,0.543252,0.00,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LMXB_91_obs_0,LMXB_91,1,XB_1732-304__________,2CXO J173547.0-302858,263.945900,-30.48282,18855.8,4.89,114.24570,LX,...,15.247567,15.146059,0.000,15.893809,0.001623,0.000211,0.735000,0.451000,0.00,5.000000
LMXB_92_obs_0,LMXB_92,1,BW_ANT_______________,2CXO J092920.1-312303,142.334100,-31.38427,24445.9,3.70,75.53454,LX,...,14.335264,14.681311,5.000,15.133535,0.000740,0.000201,0.940000,0.607000,5.09,3.970000
LMXB_94_obs_0,LMXB_94,1,J1748-3607___________,2CXO J174813.1-360758,267.054700,-36.13281,15794.5,3.09,50.50810,LX,...,14.129166,15.392119,5.000,14.971429,0.000782,0.000061,0.134000,0.380000,5.09,3.960000


In [64]:
data_id = df_comb[info_col]
data_label = df_comb['class']
data_val = df_comb.drop(columns=['class']+info_col)
data_imp_norm = df_comb.copy()
for c in data_val.columns.to_list():
    data_imp_norm.loc[:,c] = (data_imp_norm.loc[:,c] - data_imp_norm.loc[:,c].mean()) / data_imp_norm.loc[:,c].std()
data_imp_norm

,src_id,num_obs,src_n,name,ra,dec,livetime,significance,likelihood,class,...,flux_aper_hilim_u,flux_aper_lolim_h,var_inter_index_h,flux_aper_lolim_s,var_mean_b,var_sigma_b,ks_prob_b,var_prob_b,var_inter_index_s,var_inter_index_m
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,25,CSS120424:125906+242634,2CXO J004247.1+411619,10.696540,41.27215,4103.7,52.93,16618.73000,CV,...,-1.720009,0.733037,1.329855,0.440113,-0.295211,-0.008581,0.130001,0.462878,0.335040,0.619519
CV_1_obs_0,CV_1,12,W32,2CXO J002405.6-720449,6.023632,-72.08036,31673.8,20.78,1834.51100,CV,...,0.029683,0.192226,0.220405,-0.432972,-0.387764,-0.097970,0.168627,0.675923,0.335040,-1.583580
CV_2_obs_0,CV_2,8,CXOGlb_J002405.2-720446,2CXO J002405.2-720446,6.021793,-72.07968,5544.8,9.72,548.75650,CV,...,0.214212,0.802584,-1.695745,0.301137,-0.392606,-0.140135,0.509724,0.389095,-1.691877,0.034363
CV_3_obs_0,CV_3,13,Cl__NGC_104_MAM_M54898,2CXO J002406.0-720456,6.025117,-72.08228,65237.5,25.99,3275.90600,CV,...,0.084365,-0.284571,0.603634,-0.780110,-0.259087,-0.195226,-0.808349,-0.953355,0.740423,1.652307
CV_4_obs_0,CV_4,13,CXOGlb_J002404.9-720455,2CXO J002404.9-720455,6.020589,-72.08210,31673.8,19.64,2291.21000,CV,...,0.316281,-0.056891,1.370094,0.117858,-0.295740,-0.117592,0.181208,0.514410,-1.691877,0.034363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LMXB_91_obs_0,LMXB_91,1,XB_1732-304__________,2CXO J173547.0-302858,263.945900,-30.48282,18855.8,4.89,114.24570,LX,...,1.447061,0.733655,-1.695745,1.046421,-0.439280,-0.241217,0.715874,0.077902,-1.691877,0.438849
LMXB_92_obs_0,LMXB_92,1,BW_ANT_______________,2CXO J092920.1-312303,142.334100,-31.38427,24445.9,3.70,75.53454,LX,...,0.049045,0.249094,0.220405,0.131394,-0.517038,-0.245097,1.618928,0.816049,0.371524,0.022228
LMXB_94_obs_0,LMXB_94,1,J1748-3607___________,2CXO J174813.1-360758,267.054700,-36.13281,15794.5,3.09,50.50810,LX,...,-0.266780,0.990205,0.220405,-0.063710,-0.513287,-0.298329,-1.931617,-0.258049,0.371524,0.018184


In [65]:
details(data_imp_norm)

________________________________________________
Sparsity in the data : 0.00
Data shape (588, 28)
Number of sources : 
CV  	  282 	 282
PL  	  232 	 232
LX  	  74 	 74


In [66]:
verbose = 0
val_acc = []
test_acc = []
rs = 999
for i in tqdm(range(32)):
    rs = int(np.random.randint(0,99999))
    #rs = 39440
    #rs = 57865
    np.random.seed(rs)
    src_list = pd.DataFrame()
    #class_to_select = ['CV' , 'LX' , 'PL']
    df = data_imp_norm.copy()
    i_val = df[info_col]
    y_val = df['class']
    x_val = df.drop(columns=['class']+info_col)
    x_train , x_test ,i_train, i_test ,  y_train , y_test = train_test_split(x_val , i_val ,y_val , test_size=0.3 , stratify=y_val, random_state=42)
    #clf = RandomForestClassifier(n_estimators=200 , )
    clf = RandomForestClassifier(n_estimators=500  , max_depth= 10 , min_samples_split= 3, min_samples_leaf= 1 , oob_score=True , random_state=42)
    #clf = AdaBoostClassifier(base_estimator=clf ,)
    oversample = SMOTE(k_neighbors=10 , n_jobs=-1)
    x_train , y_train = oversample.fit_resample(x_train , y_train)

    clf.fit(x_train , y_train)
    v_sc = clf.score(x_test , y_test)
    test_sc = clf.score(x_train, y_train)
    val_acc.append(v_sc)
    test_acc.append(test_sc)
    #if(verbose):
    print(rs ,  v_sc)

  3%|▎         | 1/32 [00:02<01:04,  2.09s/it]

52316 0.7570621468926554


  6%|▋         | 2/32 [00:04<01:01,  2.05s/it]

94483 0.7570621468926554


  9%|▉         | 3/32 [00:06<01:01,  2.12s/it]

...

86858 0.7401129943502824


 97%|█████████▋| 31/32 [00:55<00:01,  1.55s/it]

84027 0.7627118644067796


100%|██████████| 32/32 [00:57<00:00,  1.79s/it]

39636 0.7627118644067796


In [67]:
score = pd.DataFrame({
    "Test Score" : test_acc, 
    "Validation Score" : val_acc
})*100
score.describe().loc[['mean' , 'std' , 'min', 'max' ]]

,Test Score,Validation Score
mean,99.407783,74.964689
std,0.113709,1.574859
min,99.153976,71.186441
max,99.492386,77.401130
